In [11]:
import tensorflow as tf
import numpy as np
import cv2

# Load the pre-trained model
model = tf.saved_model.load('D:\\faster_rcnn_resnet50_v1_640x640_coco17_tpu-8\\saved_model')

# Load the label map
label_map = {1: 'person', 3: 'car', 18: 'dog'}  # Replace with your label map

# Load and preprocess the input image
image = cv2.imread('D:\\image.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Convert to uint8 (Fix for TypeError)
image = tf.convert_to_tensor(image, dtype=tf.uint8)

# Expand dimensions for batch processing
image = tf.expand_dims(image, axis=0)

# Run object detection
detections = model(image)

# Extract results
boxes = detections['detection_boxes'][0].numpy()
scores = detections['detection_scores'][0].numpy()
classes = detections['detection_classes'][0].numpy().astype(int)

# Set confidence threshold
threshold = 0.5  
filtered_boxes = boxes[scores >= threshold]
filtered_classes = classes[scores >= threshold]

# Convert tensor back to numpy for OpenCV drawing
image_np = image.numpy()[0]

# Draw bounding boxes and labels
for box, cls in zip(filtered_boxes, filtered_classes):
    ymin, xmin, ymax, xmax = box
    h, w, _ = image_np.shape
    ymin, xmin, ymax, xmax = int(ymin * h), int(xmin * w), int(ymax * h), int(xmax * w)

    label = label_map.get(cls, 'Unknown')
    cv2.rectangle(image_np, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
    cv2.putText(image_np, label, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

# Display the image with detections
cv2.imshow('Object Detection', image_np)
cv2.waitKey(0)
cv2.destroyAllWindows()
